In [1]:
import mesa
import random
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.modules import CanvasGrid

class CarAgent(mesa.Agent):
    """An agent representing a car looking for a parking space."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.parked = False
        self.parking_duration = 0
        self.time_to_leave = random.randint(3, 5)
        self.steps_to_park = 0

    def move(self):
        possible_steps = [cell for cell in self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False) if not self.model.is_cell_occupied(cell)]
        if possible_steps:
            new_position = self.random.choice(possible_steps)
            self.model.grid.move_agent(self, new_position)

    def try_to_park(self):
        cell_contents = self.model.grid.get_cell_list_contents([self.pos])
        for obj in cell_contents:
            if isinstance(obj, ParkingSpaceAgent) and not obj.occupied:
                obj.occupied = True
                self.parked = True
                print(f"Car with id {self.unique_id} parked after {self.steps_to_park} steps")
                break

    def leave_parking_space(self):
        if self.parked and self.parking_duration >= self.time_to_leave:
            cell_contents = self.model.grid.get_cell_list_contents([self.pos])
            for obj in cell_contents:
                if isinstance(obj, ParkingSpaceAgent):
                    obj.occupied = False
                    break
            self.parked = False
            self.parking_duration = 0
            self.time_to_leave = random.randint(3, 5)
            self.move()

    def step(self):
        if self.parked:
            self.parking_duration += 1
            self.leave_parking_space()
        else:
            self.steps_to_park += 1
            self.move()
            self.try_to_park()

class ParkingSpaceAgent(mesa.Agent):
    """An agent representing a parking space."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.occupied = False

    def step(self):
        pass  # Parking spaces don't need to do anything each step

class ParkingLotModel(mesa.Model):
    """A model representing a parking lot with cars and parking spaces."""

    def __init__(self, N_cars, N_spaces, width, height):
        self.num_agents = N_cars + N_spaces
        self.grid = mesa.space.MultiGrid(width, height, True)
        self.schedule = mesa.time.RandomActivation(self)
        self.data_collector = mesa.DataCollector(
            agent_reporters={"StepsToPark": "steps_to_park"}
        )

        # Fixed parking space locations
        parking_space_locations = [(1, 1), (9, 1), (1, 9), (7, 9), (9, 9), (5, 9), (7, 1), (3, 9), (3, 1), (5, 1)]
        if N_spaces > len(parking_space_locations):
            raise ValueError("More parking spaces requested than predefined locations available")

        # Create car agents
        for i in range(N_cars):
            car = CarAgent(i, self)
            x, y = self.find_empty()
            self.grid.place_agent(car, (x, y))
            self.schedule.add(car)

        # Create parking space agents at fixed locations
        for i in range(N_spaces):
            space = ParkingSpaceAgent(i + N_cars, self)
            self.grid.place_agent(space, parking_space_locations[i])
            self.schedule.add(space)

    def find_empty(self):
        """Finds a random empty cell."""
        empty_cells = [(x, y) for x in range(self.grid.width) for y in range(self.grid.height) if self.is_cell_empty((x, y))]
        if empty_cells:
            return random.choice(empty_cells)
        else:
            raise Exception("No empty cells available")

    def is_cell_empty(self, pos):
        """Checks if a cell is empty."""
        return len(self.grid.get_cell_list_contents(pos)) == 0

    def is_cell_occupied(self, pos):
        """Check if a cell is occupied by a parked car."""
        cell_contents = self.grid.get_cell_list_contents(pos)
        return any(isinstance(obj, CarAgent) and obj.parked for obj in cell_contents)

    def step(self):
        self.schedule.step()
        self.data_collector.collect(self)




In [2]:
model = ParkingLotModel(N_cars=10, N_spaces=10, width=10, height=10)
for i in range(10):  
    model.step()

Car with id 1 parked after 1 steps
Car with id 9 parked after 1 steps
Car with id 2 parked after 2 steps
Car with id 0 parked after 3 steps
Car with id 3 parked after 7 steps
Car with id 0 parked after 4 steps
Car with id 1 parked after 4 steps
Car with id 6 parked after 8 steps
Car with id 8 parked after 9 steps
Car with id 4 parked after 9 steps
Car with id 2 parked after 6 steps


In [3]:
# Visualization setup
def agent_portrayal(agent):
    if isinstance(agent, CarAgent):
        portrayal = {
            "Shape": "rect",
            "Color": "red" if agent.parked else "green",
            "Filled": "true",
            "Layer": 1,
            "w": 1,
            "h": 1,
            "text": str(agent.unique_id),
            "text_color": "white"
        }
    elif isinstance(agent, ParkingSpaceAgent):
        portrayal = {
            "Shape": "rect",
            "Color": "grey" if agent.occupied else "orange",
            "Filled": "true",
            "Layer": 0,
            "w": 1,
            "h": 1
        }
    return portrayal



grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)

# Set the parameters directly
N_cars = 10
N_spaces = 5
width = 10
height = 10

server = ModularServer(ParkingLotModel,
                       [grid],
                       "Parking Lot Model",
                       {"N_cars": N_cars, "N_spaces": N_spaces, "width": width, "height": height})

server.port = 8521  # The default
server.launch()

Interface starting at http://127.0.0.1:8521


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
Car with id 9 parked after 1 steps
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
Car with id 4 parked after 4 steps
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
Car with id 3 parked after 10 steps
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
Car with id 4 parked after 8 steps
{"type":"get_step","step":14}
{"type":"get_step","step":15}
Car with id 1 parked after 15 steps
{"type":"get_step","step":16}
{"type":"get_step","step":17}
Car with id 8 parked after 17 steps
{"type":"get_step","step":18}
Car with id 4 parked after 9 steps
{"type":"get_step","step":19}
Car with id 7 parked after 19 steps
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
Car with id 4 parked after 10 ste